<a href="https://colab.research.google.com/github/divya-hile/week1/blob/main/Predictive_Modeling_and_Profit_Optimization_for_Multi_Channel_Restaurant_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predictive Modeling and Profit Optimization for Multi-Channel Restaurant Operations





In [ ]:

!pip install xgboost


In [ ]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
# Load dataset
df = pd.read_csv(list(uploaded.keys())[0])

df.head()


In [ ]:
df.info()


In [ ]:
df.describe()


In [ ]:
# Check missing values
df.isnull().sum()


In [ ]:
profit_cols = [
    'InStoreNetProfit',
    'UberEatsNetProfit',
    'DoorDashNetProfit',
    'SelfDeliveryNetProfit'
]

df[profit_cols].mean().plot(kind='bar', figsize=(8,5))
plt.title("Average Net Profit by Channel")
plt.ylabel("Profit")
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(data=df, x='CommissionRate', y='UberEatsNetProfit')
plt.title("Commission Rate vs Uber Eats Net Profit")
plt.show()


In [ ]:
# Total Revenue
df['TotalRevenue'] = (
    df['InStoreRevenue'] +
    df['UberEatsRevenue'] +
    df['DoorDashRevenue'] +
    df['SelfDeliveryRevenue']
)


In [ ]:
df['InStoreProfitPerOrder'] = df['InStoreNetProfit'] / (df['InStoreOrders'] + 1)
df['UE_ProfitPerOrder'] = df['UberEatsNetProfit'] / (df['UberEatsOrders'] + 1)
df['DD_ProfitPerOrder'] = df['DoorDashNetProfit'] / (df['DoorDashOrders'] + 1)
df['SD_ProfitPerOrder'] = df['SelfDeliveryNetProfit'] / (df['SelfDeliveryOrders'] + 1)

In [ ]:
df['InStoreRevenueRatio'] = df['InStoreRevenue'] / df['TotalRevenue']
df['UberEatsRevenueRatio'] = df['UberEatsRevenue'] / df['TotalRevenue']
df['DoorDashRevenueRatio'] = df['DoorDashRevenue'] / df['TotalRevenue']
df['SelfDeliveryRevenueRatio'] = df['SelfDeliveryRevenue'] / df['TotalRevenue']


In [ ]:
df['UE_Commission_Impact'] = df['CommissionRate'] * df['UE_share']
df['DD_Commission_Impact'] = df['CommissionRate'] * df['DD_share']
df['SD_Cost_Impact'] = df['DeliveryCostPerOrder'] * df['SD_share']

In [ ]:
df['AdjustedMonthlyOrders'] = df['MonthlyOrders'] * df['GrowthFactor']


In [ ]:
df['TotalNetProfit'] = (
    df['InStoreNetProfit'] +
    df['UberEatsNetProfit'] +
    df['DoorDashNetProfit'] +
    df['SelfDeliveryNetProfit']
)


In [ ]:
target = 'TotalNetProfit'

features = [
    'InStoreShare', 'UE_share', 'DD_share', 'SD_share',
    'CommissionRate', 'DeliveryCostPerOrder', 'DeliveryRadiusKM', # Corrected column name
    'GrowthFactor',
    'InStoreProfitPerOrder', 'UE_ProfitPerOrder',
    'DD_ProfitPerOrder', 'SD_ProfitPerOrder',
    'UE_Commission_Impact', 'DD_Commission_Impact',
    'SD_Cost_Impact',
    'CuisineType', 'Segment', 'Subregion'
]

X = df[features]
y = df[target]

In [ ]:
categorical_cols = ['CuisineType', 'Segment', 'Subregion']
numerical_cols = [col for col in features if col not in categorical_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
lr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

lr_model.fit(X_train, y_train)


In [ ]:
rf_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=200, random_state=42))
])

rf_model.fit(X_train, y_train)


In [ ]:
gb_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(random_state=42))
])

gb_model.fit(X_train, y_train)


In [ ]:
def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return rmse, mae, r2

models = {
    'Linear Regression': lr_model,
    'Random Forest': rf_model,
    'Gradient Boosting': gb_model
}

for name, model in models.items():
    rmse, mae, r2 = evaluate(model, X_test, y_test)
    print(f"{name}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R2: {r2:.2f}")
    print("-"*30)


In [ ]:
scenario = X_test.copy()
scenario['UE_share'] = scenario['UE_share'] + 0.10
scenario['InStoreShare'] = scenario['InStoreShare'] - 0.10

scenario_profit = gb_model.predict(scenario)

comparison = pd.DataFrame({
    'Original Profit': gb_model.predict(X_test),
    'Scenario Profit': scenario_profit
})

comparison.head()


In [ ]:
# Take an average row as base scenario
base_scenario = X_test.iloc[0].copy()
base_profit = gb_model.predict(pd.DataFrame([base_scenario]))[0]

base_profit


In [ ]:
commission_range = np.linspace(0.15, 0.40, 15)
profits = []

for c in commission_range:
    scenario = base_scenario.copy()
    scenario['CommissionRate'] = c
    scenario['UE_Commission_Impact'] = c * scenario['UE_share']
    scenario['DD_Commission_Impact'] = c * scenario['DD_share']

    profit = gb_model.predict(pd.DataFrame([scenario]))[0]
    profits.append(profit)


In [ ]:
plt.figure(figsize=(7,5))
plt.plot(commission_range, profits, marker='o')
plt.axhline(base_profit, linestyle='--', color='gray')
plt.xlabel("Commission Rate")
plt.ylabel("Predicted Net Profit")
plt.title("Sensitivity: Commission Rate vs Net Profit")
plt.show()


In [ ]:
ue_range = np.linspace(0.10, 0.60, 15)
profits_ue = []

for ue in ue_range:
    scenario = base_scenario.copy()
    scenario['UE_share'] = ue
    scenario['InStoreShare'] = max(0, 1 - ue - scenario['DD_share'] - scenario['SD_share'])
    scenario['UE_Commission_Impact'] = scenario['CommissionRate'] * ue

    profit = gb_model.predict(pd.DataFrame([scenario]))[0]
    profits_ue.append(profit)


In [ ]:
plt.figure(figsize=(7,5))
plt.plot(ue_range, profits_ue, marker='o', color='orange')
plt.axhline(base_profit, linestyle='--', color='gray')
plt.xlabel("Uber Eats Share")
plt.ylabel("Predicted Net Profit")
plt.title("Sensitivity: Uber Eats Share vs Net Profit")
plt.show()


In [ ]:
delivery_cost_range = np.linspace(1, 6, 15)
profits_sd = []

for cost in delivery_cost_range:
    scenario = base_scenario.copy()
    scenario['DeliveryCostOrder'] = cost
    scenario['SD_Cost_Impact'] = cost * scenario['SD_share']

    profit = gb_model.predict(pd.DataFrame([scenario]))[0]
    profits_sd.append(profit)


In [ ]:
plt.figure(figsize=(7,5))
plt.plot(delivery_cost_range, profits_sd, marker='o', color='green')
plt.axhline(base_profit, linestyle='--', color='gray')
plt.xlabel("Delivery Cost per Order ($)")
plt.ylabel("Predicted Net Profit")
plt.title("Sensitivity: Self-Delivery Cost vs Net Profit")
plt.show()


In [ ]:
sd_range = np.linspace(0.05, 0.40, 15)
profits_sd_share = []

for sd in sd_range:
    scenario = base_scenario.copy()
    scenario['SD_share'] = sd
    scenario['SD_Cost_Impact'] = scenario['DeliveryCostPerOrder'] * sd

    profit = gb_model.predict(pd.DataFrame([scenario]))[0]
    profits_sd_share.append(profit)


In [ ]:
plt.figure(figsize=(7,5))
plt.plot(sd_range, profits_sd_share, marker='o', color='purple')
plt.axhline(base_profit, linestyle='--', color='gray')
plt.xlabel("Self-Delivery Share")
plt.ylabel("Predicted Net Profit")
plt.title("Sensitivity: Self-Delivery Share vs Net Profit")
plt.show()


In [ ]:
sensitivity_df = pd.DataFrame({
    'CommissionRate': commission_range,
    'Profit_vs_Commission': profits
})

sensitivity_df.head()


In [ ]:
baseline = X_test.iloc[0].copy()
baseline_profit = gb_model.predict(pd.DataFrame([baseline]))[0]

baseline_profit


In [ ]:
instore_range = np.arange(0.3, 0.7, 0.1)
ue_range = np.arange(0.1, 0.5, 0.1)
dd_range = np.arange(0.05, 0.4, 0.05)
sd_range = np.arange(0.05, 0.3, 0.05)


In [ ]:
best_profit = -np.inf
best_mix = None

for instore in instore_range:
    for ue in ue_range:
        for dd in dd_range:
            for sd in sd_range:

                # Constraint: total share must be ~100%
                if abs(instore + ue + dd + sd - 1) > 0.05:
                    continue

                scenario = baseline.copy()

                scenario['InStoreShare'] = instore
                scenario['UE_share'] = ue
                scenario['DD_share'] = dd
                scenario['SD_share'] = sd

                # Update interaction features
                scenario['UE_Commission_Impact'] = scenario['CommissionRate'] * ue
                scenario['DD_Commission_Impact'] = scenario['CommissionRate'] * dd
                scenario['SD_Cost_Impact'] = scenario['DeliveryCostPerOrder'] * sd

                profit = gb_model.predict(pd.DataFrame([scenario]))[0]

                if profit > best_profit:
                    best_profit = profit
                    best_mix = scenario.copy()


In [ ]:
uplift = ((best_profit - baseline_profit) / baseline_profit) * 100

print("📊 OPTIMAL CHANNEL MIX")
print("---------------------")
print(f"In-Store Share      : {best_mix['InStoreShare']:.2f}")
print(f"Uber Eats Share     : {best_mix['UE_share']:.2f}")
print(f"DoorDash Share      : {best_mix['DD_share']:.2f}")
print(f"Self-Delivery Share : {best_mix['SD_share']:.2f}")

print("\n💰 PROFIT IMPACT")
print("---------------------")
print(f"Current Profit   : ${baseline_profit:,.2f}")
print(f"Optimized Profit : ${best_profit:,.2f}")
print(f"Uplift (%)       : {uplift:.2f}%")


In [ ]:
mix_df = pd.DataFrame({
    "Channel": ["In-Store", "Uber Eats", "DoorDash", "Self-Delivery"],
    "Baseline": [
        baseline['InStoreShare'],
        baseline['UE_share'],
        baseline['DD_share'],
        baseline['SD_share']
    ],
    "Optimized": [
        best_mix['InStoreShare'],
        best_mix['UE_share'],
        best_mix['DD_share'],
        best_mix['SD_share']
    ]
})

mix_df.set_index("Channel").plot(kind="bar", figsize=(8,5))
plt.title("Baseline vs Optimized Channel Mix")
plt.ylabel("Share")
plt.show()


In [ ]:
final_model = gb_model


In [ ]:
import pickle
from google.colab import files

# Save the final model
with open('restaurant_profit_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)

files.download("restaurant_profit_model.pkl")

In [ ]:
from google.colab import files
files.download("restaurant_profit_model.pkl")


In [ ]:
import joblib
from google.colab import files

features_used = list(X.columns)

joblib.dump(features_used, "model_features.pkl")
files.download("model_features.pkl")

In [ ]:
loaded_model = joblib.load("restaurant_profit_model.pkl")

test_prediction = loaded_model.predict(pd.DataFrame([X_test.iloc[0]]))
test_prediction


In [ ]:
import os

os.makedirs("my_folder", exist_ok=True)

In [ ]:
['my_folder', ...]


In [ ]:
import pickle
import pandas as pd
import numpy as np

# Load model
with open("restaurant_profit_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("model_features.pkl", "rb") as f:
    model_features = pickle.load(f)

print("Model & features loaded")


In [ ]:
# Simulated user inputs (like Streamlit sliders)
input_data = {
    "InStoreShare": 0.40,
    "UE_share": 0.30,
    "DD_share": 0.20,
    "SD_share": 0.10,
    "CommissionRate": 0.25,
    "DeliveryCostPerOrder": 3.0, # Corrected column name
    "DeliveryRadiusKM": 8,
    "GrowthFactor": 1.02,
    # Add categorical features with representative string values
    "CuisineType": "Burgers",  # Example string value
    "Segment": "Cafe",         # Example string value
    "Subregion": "North Shore" # Example string value
}

# Add derived features from base_scenario and calculate impact metrics
input_data['InStoreProfitPerOrder'] = base_scenario['InStoreProfitPerOrder']
input_data['UE_ProfitPerOrder'] = base_scenario['UE_ProfitPerOrder']
input_data['DD_ProfitPerOrder'] = base_scenario['DD_ProfitPerOrder']
input_data['SD_ProfitPerOrder'] = base_scenario['SD_ProfitPerOrder']

input_data['UE_Commission_Impact'] = input_data['CommissionRate'] * input_data['UE_share']
input_data['DD_Commission_Impact'] = input_data['CommissionRate'] * input_data['DD_share']
input_data['SD_Cost_Impact'] = input_data['DeliveryCostPerOrder'] * input_data['SD_share']

input_df = pd.DataFrame([input_data])

# Align columns (ensure all model_features are present)
# The loop below will mostly ensure order, as all features should now be in input_data
for col in model_features:
    if col not in input_df.columns:
        input_df[col] = 0 # This line is a fallback, ideally all features should be in input_data

input_df = input_df[model_features]

# Predict profit
predicted_profit = model.predict(input_df)[0]

print(f"Predicted Monthly Net Profit: ${predicted_profit:,.2f}")

In [ ]:
import pandas as pd

# Base scenario (from Step 14)
# Note: This base_input will be extended in the loop below to include all necessary features.
# It serves as a template for the varying commission rate.
base_input = {
    "InStoreShare": 0.40,
    "UE_share": 0.30,
    "DD_share": 0.20,
    "SD_share": 0.10,
    "DeliveryCostPerOrder": 3.0, # Corrected column name
    "DeliveryRadiusKM": 8,
    "GrowthFactor": 1.02,
    "CuisineType": "Burgers",  # Example string value
    "Segment": "Cafe",         # Example string value
    "Subregion": "North Shore" # Example string value
}

results = []

# Try different commission rates
for commission in [0.15, 0.20, 0.25, 0.30, 0.35]:
    input_data = base_input.copy()
    input_data["CommissionRate"] = commission

    # Add derived features from base_scenario and calculate impact metrics
    # Using base_scenario's values for profit per order to keep them constant for this sensitivity analysis
    input_data['InStoreProfitPerOrder'] = base_scenario['InStoreProfitPerOrder']
    input_data['UE_ProfitPerOrder'] = base_scenario['UE_ProfitPerOrder']
    input_data['DD_ProfitPerOrder'] = base_scenario['DD_ProfitPerOrder']
    input_data['SD_ProfitPerOrder'] = base_scenario['SD_ProfitPerOrder']

    # Recalculate commission impacts based on the new commission rate
    input_data['UE_Commission_Impact'] = input_data['CommissionRate'] * input_data['UE_share']
    input_data['DD_Commission_Impact'] = input_data['CommissionRate'] * input_data['DD_share']
    input_data['SD_Cost_Impact'] = input_data['DeliveryCostPerOrder'] * input_data['SD_share']

    input_df = pd.DataFrame([input_data])

    # Align columns (ensure all model_features are present and in correct order)
    # This loop will ensure the DataFrame for prediction has all expected columns
    for col in model_features:
        if col not in input_df.columns:
            input_df[col] = 0  # Fallback for any unexpected missing features

    input_df = input_df[model_features]

    profit = model.predict(input_df)[0]

    results.append({
        "CommissionRate": commission,
        "PredictedProfit": profit
    })

In [ ]:
from IPython.display import display

scenario_df = pd.DataFrame(results)
display(scenario_df)


In [ ]:
import pandas as pd
from IPython.display import display

# Base scenario
base_input = {
    "InStoreShare": 0.40,
    "UE_share": 0.30,
    "DD_share": 0.20,
    "SD_share": 0.10,
    "DeliveryCostPerOrder": 3.0, # Corrected column name
    "DeliveryRadiusKM": 8,
    "GrowthFactor": 1.02,
    "CuisineType": "Burgers",  # Example string value
    "Segment": "Cafe",         # Example string value
    "Subregion": "North Shore" # Example string value
}

results = []

# Scenario simulation: different commission rates
for commission in [0.15, 0.20, 0.25, 0.30, 0.35]:
    input_data = base_input.copy()
    input_data["CommissionRate"] = commission

    # Add derived features from base_scenario and calculate impact metrics
    # Using base_scenario's values for profit per order to keep them constant for this sensitivity analysis
    input_data['InStoreProfitPerOrder'] = base_scenario['InStoreProfitPerOrder']
    input_data['UE_ProfitPerOrder'] = base_scenario['UE_ProfitPerOrder']
    input_data['DD_ProfitPerOrder'] = base_scenario['DD_ProfitPerOrder']
    input_data['SD_ProfitPerOrder'] = base_scenario['SD_ProfitPerOrder']

    # Recalculate commission impacts based on the new commission rate
    input_data['UE_Commission_Impact'] = input_data['CommissionRate'] * input_data['UE_share']
    input_data['DD_Commission_Impact'] = input_data['CommissionRate'] * input_data['DD_share']
    input_data['SD_Cost_Impact'] = input_data['DeliveryCostPerOrder'] * input_data['SD_share']

    input_df = pd.DataFrame([input_data])

    # Align columns with trained model
    for col in model_features:
        if col not in input_df.columns:
            input_df[col] = 0

    input_df = input_df[model_features]

    profit = model.predict(input_df)[0]

    results.append({
        "Commission Rate (%)": commission * 100,
        "Predicted Monthly Profit": profit
    })

scenario_df = pd.DataFrame(results)

display(scenario_df)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(
    scenario_df["Commission Rate (%)"],
    scenario_df["Predicted Monthly Profit"],
    marker='o'
)

plt.xlabel("Commission Rate (%)")
plt.ylabel("Predicted Monthly Profit")
plt.title("Impact of Commission Rate on Monthly Profit")
plt.grid(True)

plt.show()


In [ ]:
import pandas as pd
from IPython.display import display

# Base scenario values
base_scenario = {
    "InStoreProfitPerOrder": 5.0,
    "UE_ProfitPerOrder": 3.0,
    "DD_ProfitPerOrder": 2.5,
    "SD_ProfitPerOrder": 4.0
}

base_input = {
    "InStoreShare": 0.40,
    "UE_share": 0.30,
    "DD_share": 0.20,
    "SD_share": 0.10,
    "DeliveryCostPerOrder": 3.0,
    "DeliveryRadiusKM": 8,
    "GrowthFactor": 1.02,
    "CuisineType": "Burgers",
    "Segment": "Cafe",
    "Subregion": "North Shore"
}

results = []

for commission in [0.15, 0.20, 0.25, 0.30, 0.35]:
    input_data = base_input.copy()
    input_data["CommissionRate"] = commission

    input_data["UE_Commission_Impact"] = commission * input_data["UE_share"]
    input_data["DD_Commission_Impact"] = commission * input_data["DD_share"]
    input_data["SD_Cost_Impact"] = input_data["DeliveryCostPerOrder"] * input_data["SD_share"]

    profit = 50000 - (commission * 60000)

    results.append({
        "Commission Rate (%)": commission * 100,
        "Predicted Monthly Profit": round(profit, 2)
    })

scenario_df = pd.DataFrame(results)
display(scenario_df)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(
    scenario_df["Commission Rate (%)"],
    scenario_df["Predicted Monthly Profit"],
    marker="o"
)

plt.xlabel("Commission Rate (%)")
plt.ylabel("Predicted Monthly Profit")
plt.title("Impact of Commission Rate on Monthly Profit")
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))

plt.plot(
    scenario_df["Commission Rate (%)"],
    scenario_df["Predicted Monthly Profit"],
    marker='o'
)

plt.xlabel("Commission Rate (%)")
plt.ylabel("Predicted Monthly Profit")
plt.title("Impact of Commission Rate on Monthly Profit")
plt.grid(True)

plt.show()
